In [4]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Literal,Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage,HumanMessage
from pydantic import BaseModel,Field
from langgraph.graph.message import  add_messages
from langgraph.checkpoint.memory import MemorySaver


In [5]:
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]

In [6]:
llm = ChatOpenAI()
def chat_node(state:ChatState):
    messages=state['messages']
    response=llm.invoke(messages)
    return {"messages":[response]}

In [7]:
checkpointer=MemorySaver()

graph=StateGraph(ChatState)

graph.add_node('chat_node',chat_node)
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot=graph.compile(checkpointer=checkpointer)


In [8]:
initial_state={
    "messages":[HumanMessage(content="Hello, how are you?")]
}
result= chatbot.invoke(initial_state)

In [ ]:
print(result['messages'][-1].content)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [ ]:
thread_id ='1'

while True:
    user_message= input('Type here: ')
    if user_message.strip().lower in ['exit','bye','quit']:
        break
    config={"configurable":{'thread_id':thread_id}}
    response =chatbot.invoke({"messages":[HumanMessage(content=user_message)]},config=config)
    print(response['messages'][-1].content)
    

In [12]:
(chatbot.get_state(config=config))

StateSnapshot(values={'messages': [HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='30e15bb6-5d9b-4f02-8b5f-520df85ee5b6'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 7, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CHrKOKhKIaL3J2JqnhmT5D2wJ4NNL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9f96824d-af1d-4c1c-99d7-efd7cca5d256-0', usage_metadata={'input_tokens': 7, 'output_tokens': 9, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content='hi', additiona